<a href="https://colab.research.google.com/github/LucasVD23/T1_AM1/blob/main/T1_Melhores_Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.colors as cl
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.utils import shuffle

In [ ]:
data_cancer = shuffle(pd.read_csv("/content/dataR2.csv"),random_state = 15)

In [ ]:
data_cancer.head()
corrs = data_cancer.corr()['Classification']
data_cancer = data_cancer.drop(columns=['Insulin','HOMA','Leptin','Adiponectin', 'MCP.1'])
#df_X = SelectKBest(score_func=chi2,k=4).fit(data_cancer[data_cancer.columns[0:9]],data_cancer[data_cancer.columns[9:10]])
#print(df_X.get_support(indices = False))
print(corrs)

Age              -0.043555
BMI              -0.132586
Glucose           0.384315
Insulin           0.276804
HOMA              0.284012
Leptin           -0.001078
Adiponectin      -0.019490
Resistin          0.227310
MCP.1             0.091381
Classification    1.000000
Name: Classification, dtype: float64


# Testando Modelos

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import metrics


In [ ]:
previsores = np.array(data_cancer.iloc[:,0:4].values)
print(previsores.shape)
classes = np.array(data_cancer.iloc[:,4:5].values)
#previsores = df_X.fit_transform(previsores,classes)

(116, 4)


In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
previsores = scaler.fit_transform(previsores)
classes = np.reshape(classes, (classes.shape[0],))

In [ ]:
for i in range(0,len(classes)):
    if(classes[i] == 1 ):
        classes[i] = 0
    else:
        classes[i] = 1
    

In [ ]:
dtc = DecisionTreeClassifier  (ccp_alpha=0.0, class_weight=None, criterion='gini',
		                       max_depth=14, max_features=None, max_leaf_nodes=None,
		                       min_impurity_decrease=0.0, min_impurity_split=None,
		                       min_samples_leaf=4, min_samples_split=2,
		                       min_weight_fraction_leaf=0.0, presort='deprecated',
		                       random_state=None, splitter='best')
                            
rfc = RandomForestClassifier  (bootstrap=True, ccp_alpha=0.0, class_weight=None,
		                       criterion='entropy', max_depth=16, max_features='auto',
		                       max_leaf_nodes=None, max_samples=None,
		                       min_impurity_decrease=0.0, min_impurity_split=None,
		                       min_samples_leaf=4, min_samples_split=2,
		                       min_weight_fraction_leaf=0.0, n_estimators=100,
		                       n_jobs=None, oob_score=False, random_state=None,
		                       verbose=0, warm_start=False)
                            
knnc = KNeighborsClassifier   (algorithm='auto', leaf_size=30, metric='minkowski',
		                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
		                     weights='distance')
                            
svmc = svm.SVC                (C=1e-06, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
		    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
		    max_iter=-1, probability=False, random_state=None, shrinking=True,
		    tol=0.001, verbose=False)
                            
adac = AdaBoostClassifier     (algorithm='SAMME.R', base_estimator=None, learning_rate=0.4,
		                   n_estimators=11, random_state=None)

In [ ]:
classifiers = {'Decision Tree': dtc, 'RandomForest': rfc, 'KNN': knnc, 'SVM': svmc, 'AdaBoost': adac}
fb_scorer = metrics.make_scorer(metrics.fbeta_score,beta=1.5)
kf = KFold(n_splits=5,shuffle = False)
for train_index, test_index in kf.split(previsores):
  print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [ 24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41
  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
 114 115] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115] TEST: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16

In [ ]:
cfs_fbetas = {}
for i in classifiers:
  print("-----------------------" + i + "---------------------")
  cf = classifiers[i]
  acuracias = []
  revocs = []
  fb_scores =[]
  precisao=[]
  especificidade=[]
  auc=[]

  for train_index, test_index in kf.split(previsores):
      X_train, X_test = previsores[train_index], previsores[test_index]
      y_train, y_test = classes[train_index], classes[test_index]
      y_train = np.reshape(y_train, (y_train.shape[0],))
      cf.fit(X_train, y_train)
      y_pred = cf.predict(X_test)

      TN, FP, FN, TP = metrics.confusion_matrix(y_test, y_pred).ravel()

      #Acuracia — (TP + TN) / total
      acuracias.append(metrics.accuracy_score(y_test,y_pred))
      #Revocaçao — TP / (TP + FN)
      revocs.append(metrics.recall_score(y_test,y_pred))
      #Precissao — TP / (TP + FP)
      precisao.append(metrics.precision_score(y_test,y_pred))
      #Especificidade — TN / (TN + FP)
      especificidade.append(TN / (TN + FP))
      
      fb_scores.append(metrics.fbeta_score(y_test,y_pred,1.5))
      auc.append(metrics.roc_auc_score(y_test, y_pred))

  print("média das acuracias: " + str(np.mean(acuracias)))
  print("média das revocações: " + str(np.mean(revocs)))
  print("média das precisões: " + str(np.mean(precisao)))
  print("média das especificidades: " + str(np.mean(especificidade)))
  print("média das f_betas: " + str(np.mean(fb_scores)))
  print("média das AUCs: " + str(np.mean(auc)))
  '''
  plt.plot(cf,acuracias)
  plt.title("Acuracias")
  plt.show()

  plt.plot(cf,revocacao)
  plt.title("Revocações")
  plt.show()

  plt.plot(cf,fb_scores)
  plt.title("FBetas")
  plt.show()
  '''
  cfs_fbetas[i] = np.mean(fb_scores)

-----------------------Decision Tree---------------------
média das acuracias: 0.7753623188405798
média das revocações: 0.819047619047619
média das precisões: 0.7919247419247418
média das especificidades: 0.7411111111111112
média das f_betas: 0.8020476026434444
média das AUCs: 0.780079365079365
-----------------------RandomForest---------------------
média das acuracias: 0.7489130434782609
média das revocações: 0.8742857142857143
média das precisões: 0.7512393162393163
média das especificidades: 0.6563888888888888
média das f_betas: 0.8077420767899774
média das AUCs: 0.7653373015873016
-----------------------KNN---------------------
média das acuracias: 0.7923913043478261
média das revocações: 0.8761904761904763
média das precisões: 0.7855677655677656
média das especificidades: 0.7258333333333333
média das f_betas: 0.8314752259570086
média das AUCs: 0.8010119047619048
-----------------------SVM---------------------
média das acuracias: 0.5514492753623188
média das revocações: 1.0
média